<a href="https://colab.research.google.com/github/ignaciomz/Geocodificacion_Masiva/blob/main/Geocodificacion_CodigoPostal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geocodificar Codigos Postales
**Este codigo te da la latitud y longitud a partir de un codigo postal usando Nominatim**

Lo complejo son las distintas nominaciones de Codigo Postal. En este caso, se esta usando Letra + Número (C1846).

En fin, espero sea de ayuda.

In [ ]:
import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# Inicializa el geocodificador de Nominatim
geolocator = Nominatim(user_agent="tuapp1.0 (tumail@gmail.com)", timeout=10)

# Función para geocodificar un código postal con manejo de tiempo de espera y reintentos
def geocode_postal_code(postal_code, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode({"postalcode": postal_code})
            if location:
                return (location.latitude, location.longitude)
            else:
                return (None, None)
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error: {e}. Intento {attempt + 1} de {retries}.")
            time.sleep(5)  # Espera 5 segundos antes de reintentar
    return (None, None)

# Cargar el archivo de Excel
archivo_excel = 'turutayarchivo.xlsx'  # Reemplaza con la ruta correcta a tu archivo Excel
df = pd.read_excel(archivo_excel)

# Verifica los nombres de las columnas para identificar la columna de códigos postales
print("Columnas en el archivo Excel:", df.columns)

# Cambia 'CodigoPostal' por el nombre correcto de la columna que contiene los códigos postales
df['latitud'] = None
df['longitud'] = None

# Recorre cada código postal y geocodifica con pausas para evitar el bloqueo
guardar_cada_n_filas = 100  # Guardar cada 100 filas
for i, row in df.iterrows():
    codigo_postal = row['codpos']  # Cambia 'CodigoPostal' por el nombre correcto de la columna
    if pd.notna(codigo_postal) and pd.isna(row['latitud']) and pd.isna(row['longitud']):  # Solo procesar si aún no tiene coordenadas
        lat, lon = geocode_postal_code(codigo_postal)
        df.at[i, 'latitud'] = lat
        df.at[i, 'longitud'] = lon
        print(f"Geocodificado: {codigo_postal} -> ({lat}, {lon})")
        time.sleep(1)  # Pausa solo si se procesa exitosamente

        # Guardar el progreso cada n filas
        if i % guardar_cada_n_filas == 0:
            df.to_excel('/content/drive/MyDrive/tuarchivo.xlsx', index=False, engine='openpyxl')  # Guardado parcial
            print(f"Progreso guardado hasta la fila {i}.")

# Guardar los resultados finales a un nuevo archivo Excel
df.to_excel('/content/drive/MyDrive/tuarchivofinal.xlsx', index=False, engine='openpyxl')  # Reemplaza con la ruta donde guardar el archivo final
print("Geocodificación completa y guardada.")